In [11]:
%load_ext autoreload
%autoreload 2

import asyncio, copy, os, socket, sys, time
from functools import partial
from multiprocessing import Pool, Process
from pathlib import Path
from tqdm import tqdm

import torch
from torch import optim
from torch.utils.tensorboard import SummaryWriter

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../../")))
from libs import agg, data, fl, hdc, log, nn, plot, poison, resnet, sim, wandb
from libs.distributed import *
from cfgs.fedargs import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
project = 'fl-hdc'
name = 'mnist'

#Define Custom CFGs
fedargs.epochs = 2
fedargs.dataset = 'fmnist'
fedargs.one_d_len = 784

# Save Logs To File (info | debug | warning | error | critical) [optional]
log.init("info")
wb = wandb.init(name, project)

wandb: wandb version 0.12.17 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [13]:
use_cuda = fedargs.cuda and torch.cuda.is_available()
torch.manual_seed(fedargs.seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

In [14]:
# Initialize Global and Client models
fedargs.model = hdc.HDC(fedargs.one_d_len, fedargs.hdc_proj_len, len(fedargs.labels), device)
hdc_model = copy.deepcopy(fedargs.model)
# Load Data to clients
train_data, test_data = data.load_dataset(fedargs.dataset, only_to_tensor = True)
train_data = data.random_split(train_data, 0.5)[0]
train_data = data.random_split(train_data, 0.5)[0]
train_data = data.random_split(train_data, 0.5)[0]

In [15]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=len(train_data), shuffle=True, num_workers=4, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=len(test_data), shuffle=True, num_workers=4, pin_memory=True)

In [16]:
%%time
train_report = hdc_model.train(train_loader, device)
log.info("\n"+train_report)

2022-05-30 12:24:33,703 - <timed exec>::<module>(l:2) : 
              precision    recall  f1-score   support

           0       0.70      0.80      0.74       787
           1       0.98      0.91      0.94       791
           2       0.55      0.63      0.59       770
           3       0.69      0.84      0.76       724
           4       0.51      0.52      0.52       747
           5       0.51      0.22      0.31       745
           6       0.39      0.25      0.30       738
           7       0.63      0.84      0.72       714
           8       0.92      0.83      0.87       712
           9       0.68      0.84      0.75       772

    accuracy                           0.67      7500
   macro avg       0.66      0.67      0.65      7500
weighted avg       0.66      0.67      0.65      7500
 [MainProcess : MainThread (INFO)]


CPU times: user 18.3 s, sys: 17 s, total: 35.4 s
Wall time: 3.47 s


In [17]:
%%time
test_report = hdc_model.test(test_loader, device)
log.info("\n"+test_report)

2022-05-30 12:24:36,488 - <timed exec>::<module>(l:2) : 
              precision    recall  f1-score   support

           0       0.67      0.77      0.72      1000
           1       0.99      0.86      0.92      1000
           2       0.53      0.61      0.57      1000
           3       0.70      0.82      0.76      1000
           4       0.50      0.51      0.50      1000
           5       0.50      0.21      0.30      1000
           6       0.35      0.23      0.28      1000
           7       0.66      0.87      0.75      1000
           8       0.89      0.84      0.87      1000
           9       0.67      0.85      0.75      1000

    accuracy                           0.66     10000
   macro avg       0.65      0.66      0.64     10000
weighted avg       0.65      0.66      0.64     10000
 [MainProcess : MainThread (INFO)]


CPU times: user 20.3 s, sys: 17.9 s, total: 38.2 s
Wall time: 2.74 s


In [20]:
for epoch in range(fedargs.epochs):
    train_report = hdc_model.re_train(train_loader, device)
    test_report = hdc_model.test(test_loader, device)
    log.info("\n"+test_report)

tensor([[0.9999, 0.9997, 0.9999,  ..., 0.9996, 0.9999, 0.9998],
        [0.9992, 0.9990, 0.9993,  ..., 0.9996, 0.9995, 0.9995],
        [0.9998, 0.9998, 0.9998,  ..., 0.9994, 0.9997, 0.9996],
        ...,
        [0.9992, 0.9990, 0.9993,  ..., 0.9993, 0.9994, 0.9995],
        [0.9999, 0.9999, 0.9999,  ..., 0.9995, 0.9998, 0.9996],
        [0.9995, 0.9993, 0.9995,  ..., 0.9991, 0.9995, 0.9994]])
[tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.])]
[tensor([0.9998]), tensor([0.9999]), tensor([0.9997]), tensor([0.9999]), tensor([0.9998]), tensor([0.9994]), tensor([0.9997]), tensor([0.9993]), tensor([0.9996]), tensor([0.9995])]
[tensor([0.9999]), tensor([0.9997]), tensor([1.0000]), tensor([0.9999]), tensor([1.0000]), tensor([0.9998]), tensor([1.0000]), tensor([0.9996]), tensor([0.9999]), tensor([0.9998])]
[tensor([0.9999]), tensor([0.9999]), tensor([0.9999]), tensor([1.0000]), tensor([0.9999]), tensor([

[tensor([0.9995]), tensor([0.9993]), tensor([0.9996]), tensor([0.9995]), tensor([0.9996]), tensor([0.9999]), tensor([0.9996]), tensor([0.9999]), tensor([0.9998]), tensor([0.9999])]
[tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.])]
[tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.])]


2022-05-30 12:27:29,429 - <ipython-input-20-12bad473728f>::<module>(l:4) : 
              precision    recall  f1-score   support

           0       0.67      0.81      0.73      1000
           1       0.96      0.90      0.93      1000
           2       0.62      0.73      0.67      1000
           3       0.77      0.81      0.79      1000
           4       0.62      0.73      0.67      1000
           5       0.89      0.62      0.73      1000
           6       0.57      0.27      0.36      1000
           7       0.79      0.89      0.83      1000
           8       0.87      0.88      0.88      1000
           9       0.78      0.90      0.83      1000

    accuracy                           0.75     10000
   macro avg       0.75      0.75      0.74     10000
weighted avg       0.75      0.75      0.74     10000
 [MainProcess : MainThread (INFO)]


tensor([[0.9991, 0.9989, 0.9991,  ..., 0.9997, 0.9993, 0.9993],
        [0.9994, 0.9992, 0.9995,  ..., 0.9998, 0.9996, 0.9997],
        [0.9989, 0.9991, 0.9989,  ..., 0.9983, 0.9987, 0.9985],
        ...,
        [0.9999, 0.9998, 1.0000,  ..., 0.9996, 0.9999, 0.9998],
        [0.9999, 0.9997, 1.0000,  ..., 0.9996, 0.9999, 0.9998],
        [0.9997, 0.9999, 0.9997,  ..., 0.9993, 0.9995, 0.9995]])
[tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.])]
[tensor([0.9999]), tensor([0.9999]), tensor([0.9998]), tensor([0.9999]), tensor([0.9998]), tensor([0.9995]), tensor([0.9998]), tensor([0.9994]), tensor([0.9997]), tensor([0.9996])]
[tensor([1.0000]), tensor([0.9998]), tensor([1.0000]), tensor([0.9999]), tensor([1.0000]), tensor([0.9998]), tensor([1.0000]), tensor([0.9996]), tensor([0.9999]), tensor([0.9998])]
[tensor([0.9999]), tensor([0.9999]), tensor([0.9999]), tensor([1.0000]), tensor([0.9999]), tensor([

[tensor([0.9995]), tensor([0.9993]), tensor([0.9996]), tensor([0.9995]), tensor([0.9996]), tensor([0.9999]), tensor([0.9996]), tensor([0.9999]), tensor([0.9998]), tensor([0.9999])]
[tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.])]
[tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.]), tensor([0.])]


2022-05-30 12:28:08,422 - <ipython-input-20-12bad473728f>::<module>(l:4) : 
              precision    recall  f1-score   support

           0       0.69      0.81      0.74      1000
           1       0.94      0.91      0.93      1000
           2       0.52      0.57      0.54      1000
           3       0.80      0.80      0.80      1000
           4       0.65      0.51      0.57      1000
           5       0.87      0.71      0.78      1000
           6       0.43      0.42      0.43      1000
           7       0.83      0.86      0.84      1000
           8       0.87      0.89      0.88      1000
           9       0.80      0.91      0.85      1000

    accuracy                           0.74     10000
   macro avg       0.74      0.74      0.74     10000
weighted avg       0.74      0.74      0.74     10000
 [MainProcess : MainThread (INFO)]


In [19]:
a = torch.tensor([1,2,3])
a.tolist()

[1, 2, 3]